In [ ]:
import pandas as pd
import pickle
data = pd.read_feather('../Data/df_cleaned_2')

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from collections import defaultdict
import re
import spacy
import fr_core_news_md
from bs4 import BeautifulSoup
import sys
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, concatenate, Activation, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier, KerasRegressor

import matplotlib.pyplot as plt
plt.switch_backend('agg')
get_ipython().run_line_magic('matplotlib', 'inline')


from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_feather('../Data/df_cleaned_2')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42825 entries, 0 to 42824
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            42825 non-null  int64         
 1   subject_id       42825 non-null  int64         
 2   hadm_id          42825 non-null  int64         
 3   admittime        42825 non-null  datetime64[ns]
 4   dischtime        42825 non-null  datetime64[ns]
 5   first_careunit   42825 non-null  object        
 6   last_careunit    42825 non-null  object        
 7   age              42825 non-null  float64       
 8   gender           42825 non-null  object        
 9   marital_status   41192 non-null  object        
 10  insurance        42825 non-null  object        
 11  diagnosis        42824 non-null  object        
 12  text             42825 non-null  object        
 13  next_readmit_dt  42825 non-null  float64       
 14  target           42825 non-null  int32

In [4]:
data.head(3)

,index,subject_id,hadm_id,admittime,dischtime,first_careunit,last_careunit,age,gender,marital_status,insurance,diagnosis,text,next_readmit_dt,target,clean
0,0,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,MICU,MICU,76.526788,M,MARRIED,Medicare,HYPOTENSION,Admission Date: [**2101-10-20**] Discharg...,0.0,0,admit hypotension urine illness hospitalize ho...
1,1,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,MICU,MICU,47.845044,F,SINGLE,Private,"FEVER,DEHYDRATION,FAILURE TO THRIVE",Admission Date: [**2191-3-16**] Discharge...,0.0,0,shortness breath fever illness human immunodef...
2,2,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,SICU,SICU,65.940670,F,MARRIED,Medicare,CHRONIC RENAL FAILURE/SDA,Admission Date: [**2175-5-30**] Dischar...,0.0,0,end stage renal disease admit transplant surge...


In [5]:
df = data[["diagnosis", "clean", "target"]]

In [6]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300

In [7]:
macronum=sorted(set(df['target']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

In [8]:
df['target']=df['target'].apply(fun)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42825 entries, 0 to 42824
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   diagnosis  42824 non-null  object
 1   clean      42825 non-null  object
 2   target     42825 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1003.8+ KB


<ipython-input-8-75c5d8457749>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target']=df['target'].apply(fun)


In [9]:
labels = []

for idx in df['target']:
    labels.append(idx)

In [10]:
def pre_proc(df, text_name) :

    texts = []

    for idx in range(df[text_name].shape[0]):
        texts.append(df[text_name][idx])
    
    # Transformation des tokens des textes en sequences de tokens
    
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    word_index = tokenizer.word_index

    return seq, df, word_index

In [11]:
seq, df, word_index = pre_proc(df, 'clean')

print('Number of Unique Tokens',len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of Data Sequence Tensor:', seq.shape)
print('Shape of Label Tensor:', labels.shape)

Number of Unique Tokens 202601
Shape of Data Sequence Tensor: (42825, 1000)
Shape of Label Tensor: (42825, 2)


In [12]:
X_train_full, X_test, y_train_full, y_test = train_test_split(seq, labels, test_size = 0.2, stratify = labels)

In [13]:
embeddings_index = {}
f = open('F:/Dropbox/Travaux_JAOTOMBO/These_Sante_Publique/NLP/Embeddings/fasttext_fr/cc.fr.300.vec/cc.fr.300.vec',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in FastText 4GB.' % len(embeddings_index))

Total 2000000 word vectors in FastText 4GB.


In [14]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [15]:
def create_model() :
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    bigram_branch = Conv1D(filters=128, kernel_size=2, padding='valid', activation='relu', strides=1)(embedded_sequences)
    bigram_branch = GlobalMaxPooling1D()(bigram_branch)
    
    trigram_branch = Conv1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1)(embedded_sequences)
    trigram_branch = GlobalMaxPooling1D()(trigram_branch)
    
    fourgram_branch = Conv1D(filters=128, kernel_size=4, padding='valid', activation='relu', strides=1)(embedded_sequences)
    fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
    
    fivegram_branch = Conv1D(filters=128, kernel_size=4, padding='valid', activation='relu', strides=1)(embedded_sequences)
    fivegram_branch = GlobalMaxPooling1D()(fivegram_branch)
    
    merged = concatenate([bigram_branch, trigram_branch, fourgram_branch, fivegram_branch], axis=1)
    
    merged = Dense(256, activation='relu')(merged)
    merged = Dropout(0.2)(merged)
    merged = Dense(len(macronum))(merged)
    
    output = Activation('sigmoid')(merged)
    model = Model(inputs=[sequence_input], outputs=[output])
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['acc',tf.keras.metrics.AUC()])
    model.summary()
    return model

In [16]:
model = KerasClassifier(build_fn=create_model)

In [21]:
# Create train and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=None, stratify=y_train_full)

# Fit the model to the training data
checkpoint_cb = ModelCheckpoint("Yoon_Kim_Model.hdf5",monitor='val_acc', verbose=1, save_best_only=True)
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=2, callbacks=[checkpoint_cb, early_stopping_cb])


In [27]:
# Compute predicted probabilities: y_pred_prob
y_pred_proba = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)[:,1]


268/268 [==============================] - 2s 8ms/step


In [28]:
# Compute scores
auc_score = roc_auc_score(y_test[:,1], y_pred_proba)
accuracy_score  = accuracy_score(y_test[:,1], y_pred)

In [29]:
print(accuracy_score, auc_score)

0.9064798598949212 0.6166681100101186
